<a href="https://colab.research.google.com/github/tako6969/B4_model-/blob/main/B4_finalmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

これはモデルのみ参考に、その他の操作は適当なコピペ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.layers import BatchNormalization, add
from keras.layers import Conv2D

import tensorflow as tf
from keras.layers import Conv2D, UpSampling2D
from keras.layers import add
from keras.models import Model


from keras.layers import LeakyReLU
from keras.layers import Dense, Dropout,Input,Average,Conv2DTranspose,SeparableConv2D,dot,UpSampling2D,Add, Flatten,Concatenate,Multiply,Conv2D, MaxPooling2D,Activation,AveragePooling2D, ZeroPadding2D,GlobalAveragePooling2D,multiply,DepthwiseConv2D,ZeroPadding2D,GlobalAveragePooling2D,concatenate ,Lambda


kernel_initializer = 'he_uniform'
interpolation = "nearest"


def conv_block_2D(x, filters, block_type, repeat=1, dilation_rate=1, size=3, padding='same'):
    result = x

    for i in range(0, repeat):

        if block_type == 'separated':
            result = separated_conv2D_block(result, filters, size=size, padding=padding)
        elif block_type == 'duckv2':
            result = duckv2_conv2D_block(result, filters, size=size)
        elif block_type == 'midscope':
            result = midscope_conv2D_block(result, filters)
        elif block_type == 'widescope':
            result = widescope_conv2D_block(result, filters)
        elif block_type == 'resnet':
            result = resnet_conv2D_block(result, filters, dilation_rate)
        elif block_type == 'conv':
            result = Conv2D(filters, (size, size),
                            activation='relu', kernel_initializer=kernel_initializer, padding=padding)(result)
        elif block_type == 'double_convolution':
            result = double_convolution_with_batch_normalization(result, filters, dilation_rate)

        else:
            return None

    return result


def duckv2_conv2D_block(x, filters, size):
    x = BatchNormalization(axis=-1)(x)
    x1 = widescope_conv2D_block(x, filters)

    x2 = midscope_conv2D_block(x, filters)

    x3 = conv_block_2D(x, filters, 'resnet', repeat=1)

    x4 = conv_block_2D(x, filters, 'resnet', repeat=2)

    x5 = conv_block_2D(x, filters, 'resnet', repeat=3)

    x6 = separated_conv2D_block(x, filters, size=6, padding='same')

    x = add([x1, x2, x3, x4, x5, x6])

    x = BatchNormalization(axis=-1)(x)

    return x


def separated_conv2D_block(x, filters, size=3, padding='same'):
    x = Conv2D(filters, (1, size), activation='relu', kernel_initializer=kernel_initializer, padding=padding)(x)

    x = BatchNormalization(axis=-1)(x)

    x = Conv2D(filters, (size, 1), activation='relu', kernel_initializer=kernel_initializer, padding=padding)(x)

    x = BatchNormalization(axis=-1)(x)

    return x


def midscope_conv2D_block(x, filters):
    x = Conv2D(filters, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same',
               dilation_rate=1)(x)

    x = BatchNormalization(axis=-1)(x)

    x = Conv2D(filters, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same',
               dilation_rate=2)(x)

    x = BatchNormalization(axis=-1)(x)

    return x


def widescope_conv2D_block(x, filters):
    x = Conv2D(filters, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same',
               dilation_rate=1)(x)

    x = BatchNormalization(axis=-1)(x)

    x = Conv2D(filters, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same',
               dilation_rate=2)(x)

    x = BatchNormalization(axis=-1)(x)

    x = Conv2D(filters, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same',
               dilation_rate=3)(x)

    x = BatchNormalization(axis=-1)(x)

    return x


def resnet_conv2D_block(x, filters, dilation_rate=1):
    x1 = Conv2D(filters, (1, 1), activation='relu', kernel_initializer=kernel_initializer, padding='same',
                dilation_rate=dilation_rate)(x)

    x = Conv2D(filters, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same',
               dilation_rate=dilation_rate)(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same',
               dilation_rate=dilation_rate)(x)
    x = BatchNormalization(axis=-1)(x)
    x_final = add([x, x1])

    x_final = BatchNormalization(axis=-1)(x_final)

    return x_final


def double_convolution_with_batch_normalization(x, filters, dilation_rate=1):
    x = Conv2D(filters, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same',
               dilation_rate=dilation_rate)(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same',
               dilation_rate=dilation_rate)(x)
    x = BatchNormalization(axis=-1)(x)

    return x

def TSFB(x,y,nf1,nf2,gc):

    x_ = Conv2D(filters=nf1, kernel_size=3, strides=1,padding='same')(x)
    x_ = LeakyReLU(alpha=0.25)(x_)
    x_ = BatchNormalization(axis=-1)(x_)

    y_ = Conv2D(filters=nf2, kernel_size=3, strides=1,padding='same')(y)
    y_ = LeakyReLU(alpha=0.25)(y_)
    y_ = BatchNormalization(axis=-1)(y_)

    x1 = Conv2D(filters=gc, kernel_size=3, strides=1,padding='same')(x)
    x1 = LeakyReLU(alpha=0.25)(x1)
    x1 = BatchNormalization(axis=-1)(x1)

    y1 = Conv2D(filters=gc, kernel_size=3, strides=1,padding='same')(y)
    y1 = LeakyReLU(alpha=0.25)(y1)
    y1 = BatchNormalization(axis=-1)(y1)

    x1c = Conv2D(filters=gc, kernel_size=3, strides=2,padding='same')(x)
    x1c = LeakyReLU(alpha=0.25)(x1c)
    x1c = BatchNormalization(axis=-1)(x1c)

    y1t = Conv2DTranspose(filters=gc, kernel_size=3, strides=2,padding='same')(y)
    y1t = LeakyReLU(alpha=0.25)(y1t)
    y1t = BatchNormalization(axis=-1)(y1t)

    x2_input = concatenate([x,x1,y1t],axis=-1)
    x2 = Conv2D(filters= nf1, kernel_size=3,strides=1, padding='same')(x2_input)
    x2 = LeakyReLU(alpha=0.25)(x2)
    x2 = BatchNormalization(axis=-1)(x2)

    y2_input = concatenate([y,y1,x1c],axis=-1)
    y2 = Conv2D(filters= nf2, kernel_size=3,strides=1, padding='same')(y2_input)
    y2 = LeakyReLU(alpha=0.25)(y2)
    y2 = BatchNormalization(axis=-1)(y2)

    x5 = Lambda(lambda x: x * 0.4)(x2)
    y5 = Lambda(lambda x: x * 0.4)(y2)

    out1 = add([x5,x_])
    out2 = add([y5,y_])

    out1 = BatchNormalization(axis=-1)(out1)
    out2 = BatchNormalization(axis=-1)(out2)

    return out1, out2


def ChannelAttentionModule(input: tf.keras.Model, ratio=8):
    channel = input.shape[-1]

    shared_dense_one = tf.keras.layers.Dense(channel // ratio,
                                             activation='relu',
                                             kernel_initializer='he_normal',
                                             use_bias=True,
                                             bias_initializer='zeros')
    shared_dense_two = tf.keras.layers.Dense(channel,
                                             kernel_initializer='he_normal',
                                             use_bias=True,
                                             bias_initializer='zeros')

    avg_pool = tf.keras.layers.GlobalAveragePooling2D()(input)
    avg_pool = tf.keras.layers.Reshape((1, 1, channel))(avg_pool)
    avg_pool = shared_dense_one(avg_pool)
    avg_pool = shared_dense_two(avg_pool)

    max_pool = tf.keras.layers.GlobalMaxPooling2D()(input)
    max_pool = tf.keras.layers.Reshape((1, 1, channel))(max_pool)
    max_pool = shared_dense_one(max_pool)
    max_pool = shared_dense_two(max_pool)

    x = tf.keras.layers.Add()([avg_pool, max_pool])
    x = tf.keras.layers.Activation('sigmoid')(x)

    return tf.keras.layers.multiply([input, x])

def SpatialAttentionModule(input: tf.keras.Model, kernel_size=7):
    avg_pool = tf.keras.layers.Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(input)
    max_pool = tf.keras.layers.Lambda(lambda x: K.max(x, axis=3, keepdims=True))(input)
    x = tf.keras.layers.Concatenate(axis=3)([avg_pool, max_pool])
    for i in [64, 32, 16]:
        x = tf.keras.layers.Conv2D(filters=i,
                                   kernel_size=kernel_size,
                                   strides=1,
                                   padding='same',
                                   activation='relu',
                                   kernel_initializer='he_normal',
                                   use_bias=False)(x)
    x = tf.keras.layers.Conv2D(filters=1,
                               kernel_size=kernel_size,
                               strides=1,
                               padding='same',
                               activation='sigmoid',
                               kernel_initializer='he_normal',
                               use_bias=False)(x)

    return tf.keras.layers.multiply([input, x])


def CBAM(x):
    x = ChannelAttentionModule(x)
    x = SpatialAttentionModule(x)
    return x

In [ ]:
import tensorflow as tf
from keras.layers import Conv2D, UpSampling2D
from keras.layers import add
from keras.models import Model

kernel_initializer = 'he_uniform'
interpolation = "nearest"


def create_model(img_height, img_width, input_chanels, out_classes, starting_filters):
    input_layer = tf.keras.layers.Input((img_height, img_width, input_chanels))

    print('Starting DUCK-Net')

    p1 = Conv2D(starting_filters * 2, 2, strides=2, padding='same')(input_layer)
    p2 = Conv2D(starting_filters * 4, 2, strides=2, padding='same')(p1)
    p3 = Conv2D(starting_filters * 8, 2, strides=2, padding='same')(p2)
    p4 = Conv2D(starting_filters * 16, 2, strides=2, padding='same')(p3)
    p5 = Conv2D(starting_filters * 32, 2, strides=2, padding='same')(p4)

    t0_ = conv_block_2D(input_layer, starting_filters, 'duckv2', repeat=1)
    t0 =  CBAM(t0_)

    l1i = Conv2D(starting_filters * 2, 2, strides=2, padding='same')(t0_)
    s1 = add([l1i, p1])
    t1 = conv_block_2D(s1, starting_filters * 2, 'duckv2', repeat=1)

    l2i = Conv2D(starting_filters * 4, 2, strides=2, padding='same')(t1)
    s2 = add([l2i, p2])
    t2 = conv_block_2D(s2, starting_filters * 4, 'duckv2', repeat=1)

    l3i = Conv2D(starting_filters * 8, 2, strides=2, padding='same')(t2)
    s3 = add([l3i, p3])
    t3 = conv_block_2D(s3, starting_filters * 8, 'duckv2', repeat=1)

    l4i = Conv2D(starting_filters * 16, 2, strides=2, padding='same')(t3)
    s4 = add([l4i, p4])
    t4 = conv_block_2D(s4, starting_filters * 16, 'duckv2', repeat=1)

    l5i = Conv2D(starting_filters * 32, 2, strides=2, padding='same')(t4)
    s5 = add([l5i, p5])
    t51 = conv_block_2D(s5, starting_filters * 32, 'duckv2', repeat=1)

    b1, b2 = TSFB(t0, t1, nf1=17, nf2=34, gc=64)
    b4, b3 = TSFB(b2, t2, nf1=34, nf2=68, gc=64)
    b5, b6 = TSFB(b1, b4, nf1=17, nf2=34,gc=64)
    a2, a1 = TSFB(t4, t51, nf1=272, nf2=544, gc=64)
    a4, a3 = TSFB(t3, a2, nf1=136, nf2=272, gc=64)
    a6, a5 = TSFB(b3, a4, nf1=68, nf2=136,gc=64)
    a8, a7 = TSFB(b6, a6, nf1=34, nf2=68,gc=64)
    a10, a9 = TSFB(b5, a8, nf1=17, nf2=34,gc=64)

    t53 = conv_block_2D(a1, starting_filters * 16, 'duckv2', repeat=1)

    l5o = UpSampling2D((2, 2), interpolation=interpolation)(t53)
    c4 = add([l5o, a3])
    q4 = conv_block_2D(c4, starting_filters * 8, 'duckv2', repeat=1)

    l4o = UpSampling2D((2, 2), interpolation=interpolation)(q4)
    c3 = add([l4o, a5])
    q3 = conv_block_2D(c3, starting_filters * 4, 'duckv2', repeat=1)

    l3o = UpSampling2D((2, 2), interpolation=interpolation)(q3)
    c2 = add([l3o, a7])
    q6 = conv_block_2D(c2, starting_filters * 2, 'duckv2', repeat=1)

    l2o = UpSampling2D((2, 2), interpolation=interpolation)(q6)
    c1 = add([l2o, a9])
    q1 = conv_block_2D(c1, starting_filters, 'duckv2', repeat=1)

    l1o = UpSampling2D((2, 2), interpolation=interpolation)(q1)
    c0 = add([l1o, a10])
    z1 = conv_block_2D(c0, starting_filters, 'duckv2', repeat=1)
    z1_ = conv_block_2D(c0, starting_filters, 'duckv2', repeat=1)

    output_1 = conv_block_2D(z1, starting_filters, 'duckv2', repeat=1)
    output_2 = conv_block_2D(z1_, starting_filters, 'duckv2', repeat=1)

    output_1 = Conv2D(out_classes, (1, 1), activation='sigmoid')(output_1)
    output_2 = Conv2D(out_classes, (1, 1), activation='sigmoid')(output_2)

    output_1 = Dense(1, activation='sigmoid', name='output_1')(output_1)
    output_2 = Dense(1, activation='sigmoid', name='output_2')(output_2)

    model = Model(inputs=[input_layer], outputs=[output_1, output_2])

    return model

In [ ]:
import keras.backend as K
import tensorflow as tf
import numpy as np

def dice_metric_loss(ground_truth, predictions, smooth=1e-6):
    ground_truth = K.cast(ground_truth, tf.float32)
    predictions = K.cast(predictions, tf.float32)
    ground_truth = K.flatten(ground_truth)
    predictions = K.flatten(predictions)
    intersection = K.sum(predictions * ground_truth)
    union = K.sum(predictions) + K.sum(ground_truth)
    dice = (2. * intersection + smooth) / (union + smooth)

   #  BCE = K.binary_crossentropy(target=ground_truth, output=predictions)

    return 1 - dice


In [ ]:
import glob

import numpy as np
from PIL import Image
from skimage.io import imread
from tqdm import tqdm

folder_path = "/content/drive/MyDrive/new_dataset/"  # Add the path to your data directory


def load_data(img_height, img_width, images_to_be_loaded, dataset):
    IMAGES_PATH = folder_path + 'image/'
    MASKS_PATH = folder_path + 'lesion mask/'
    BONE_PATH = folder_path + 'bone mask/'


    if dataset == 'kvasir':
        train_ids = glob.glob(IMAGES_PATH + "*.jpg")

    if dataset == 'cvc-clinicdb':
        train_ids = glob.glob(IMAGES_PATH + "*.tif")

    if dataset == 'cvc-colondb' or dataset == 'etis-laribpolypdb':
        train_ids = glob.glob(IMAGES_PATH + "*.png")

    if images_to_be_loaded == -1:
        images_to_be_loaded = len(train_ids)

    X_train = np.zeros((images_to_be_loaded, img_height, img_width, 3), dtype=np.float32)
    Y_train = np.zeros((images_to_be_loaded, img_height, img_width), dtype=np.uint8)
    Z_train = np.zeros((images_to_be_loaded, img_height, img_width), dtype=np.uint8)

    print('Resizing training images and masks: ' + str(images_to_be_loaded))
    for n, id_ in tqdm(enumerate(train_ids)):
        if n == images_to_be_loaded:
            break

        image_path = id_
        mask_path = image_path.replace("image", "lesion mask")
        bone_path = image_path.replace("image", "bone mask")

        image = imread(image_path)
        mask_ = imread(mask_path)
        bone_ = imread(bone_path)

        mask = np.zeros((img_height, img_width), dtype=np.bool_)
        bone = np.zeros((img_height, img_width), dtype=np.bool_)

        pillow_image = Image.fromarray(image)

        pillow_image = pillow_image.resize((img_height, img_width))
        image = np.array(pillow_image)

        X_train[n] = image / 255

        pillow_mask = Image.fromarray(mask_)
        pillow_mask = pillow_mask.resize((img_height, img_width), resample=Image.LANCZOS)
        mask_ = np.array(pillow_mask)

        pillow_bone = Image.fromarray(bone_)
        pillow_bone = pillow_bone.resize((img_height, img_width), resample=Image.LANCZOS)
        bone_ = np.array(pillow_bone)

        for i in range(img_height):
            for j in range(img_width):
                if np.all(mask_[i, j] >= 127):
                    mask[i, j] = 1

        for i in range(img_height):
            for j in range(img_width):
                if np.all(bone_[i, j] >= 127):
                    bone[i, j] = 1

        Y_train[n] = mask

        Z_train[n] = bone

    Y_train = np.expand_dims(Y_train, axis=-1)

    Z_train = np.expand_dims(Z_train, axis=-1)

    return X_train, Y_train, Z_train

# X_train : Y_train : Z_train = CT : lesion : bone

In [ ]:
# Importing the necessary libraries

import tensorflow as tf
import albumentations as albu
import numpy as np
import gc
import pickle
import matplotlib.pyplot as plt
from keras.callbacks import CSVLogger
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score, precision_score, recall_score, accuracy_score, f1_score

from tensorflow import keras
from keras.optimizers import Adam

import keras.backend as K

In [ ]:
# Checking the number of GPUs available

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# フィルター17でやってるよ
# Setting the model parameters

img_size = 224
dataset_type = 'BM_Seg' # Options: kvasir/cvc-clinicdb/cvc-colondb/etis-laribpolypdb
learning_rate = 0.0001
seed_value = 58800
filters = 17 # Number of filters, the paper presents the results with 17 and 34
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

ct = datetime.now()

model_type = "DuckNet"

progress_path = '/content/ProgressFull/' + dataset_type + '_progress_csv_' + model_type + '_filters_' + str(filters) +  '_'  + str(ct) + '.csv'
progressfull_path = '/content/ProgressFull/' + dataset_type + '_progress_' + model_type + '_filters_' + str(filters) + '_' + str(ct) + '.txt'
plot_path = '/content/ProgressFull/' + dataset_type + '_progress_plot_' + model_type + '_filters_' + str(filters) + '_' + str(ct) + '.png'
model_path = '/content/ModelSaveTensorFlow/' + dataset_type + '/' + model_type + '_filters_' + str(filters) + '_' + str(ct)

EPOCHS = 200
min_loss_for_saving = 1

In [ ]:
# Loading the data

X, Y, Z = load_data(img_size, img_size, -1, 'kvasir')

In [ ]:
x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(X, Y, Z, test_size=0.1, shuffle= True, random_state = seed_value)
x_train, x_valid, y_train, y_valid, z_train, z_valid = train_test_split(x_train, y_train, z_train, test_size=0.111, shuffle= True, random_state = seed_value)

In [ ]:
import matplotlib.pyplot as plt

for i in range(152):
    print(i)
    img_1 = np.array(x_test[i])
    img_2 = np.array(y_test[i])
    img_3 = np.array(z_test[i])
    plt.imshow(img_1)
    plt.show()
    plt.imshow(img_2)
    plt.show()
    plt.imshow(img_3)
    plt.show()

In [ ]:
# Defining the augmentations

aug_train = albu.Compose(
    [
    albu.HorizontalFlip(p=0.5),
    albu.VerticalFlip(p=0.1),
    albu.Rotate(limit=35, p=1.0),
    ],
    additional_targets={'mask': 'image', 'bone': 'image'}
)

def augment_images():
    x_train_out = []
    y_train_out = []
    z_train_out = []

    for i in range (len(x_train)):
        ug = aug_train(image=x_train[i], mask=y_train[i], bone=z_train[i])
        x_train_out.append(ug['image'])
        y_train_out.append(ug['mask'])
        z_train_out.append(ug['bone'])

    return np.array(x_train_out), np.array(y_train_out), np.array(z_train_out)

In [ ]:
# Training the model
model = tf.keras.models.load_model(model_path, custom_objects={'dice_metric_loss':dice_metric_loss})

ct = datetime.now()
model_path = '/content/ModelSaveTensorFlow/' + dataset_type + '/' + model_type + '_filters_' + str(filters) + '_' + str(ct)
min_loss_for_saving = 1

learning_rate = 0.0005
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss={'output_1':dice_metric_loss, 'output_2':'binary_crossentropy'},loss_weights={'output_1':0.8, 'output_2':0.2})


step = 0
# 実験

# 一旦100でモデルを保存！
EPOCHS = 100

for epoch in range(0, EPOCHS):

    if epoch == 50:
        learning_rate = 0.0001
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss={'output_1':dice_metric_loss, 'output_2':'binary_crossentropy'},loss_weights={'output_1':0.8, 'output_2':0.2})

    print(f'Training, epoch {epoch}')
    print('Learning Rate: ' + str(learning_rate))

    step += 1

    image_augmented, mask_augmented, bone_augmented = augment_images()

    # csv_logger = CSVLogger(progress_path, append=True, separator=';')
    # log_dir = "logs/" + str(ct)
    # tb_cb = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)
    log_dir = "logs/fit/" + str(ct)
    tb_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True)

    model.fit(x= image_augmented, y={'output_1': mask_augmented, 'output_2': bone_augmented}, epochs=1, batch_size=16, validation_data=(x_valid, {'output_1': y_valid, 'output_2': z_valid}), verbose=1, callbacks=[tb_cb])

    prediction_valid, bone_predict_valid = model.predict(x_valid, batch_size=4, verbose=0)
    loss_valid = dice_metric_loss(y_valid, prediction_valid)

    loss_valid = loss_valid.numpy()
    print("Loss Validation: " + str(loss_valid))

    dice_valid = f1_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                           np.ndarray.flatten(prediction_valid > 0.5))
    print("dice_valid: " + str(dice_valid))

    # prediction_test = model.predict(x_test, verbose=0)
    # loss_test = dice_metric_loss(y_test, prediction_test)
    # loss_test = loss_test.numpy()
    # print("Loss Test: " + str(loss_test))

    # with open(progressfull_path, 'a') as f:
        # f.write('epoch: ' + str(epoch) + '\nval_loss: ' + str(loss_valid) + '\ntest_loss: ' + str(loss_test) + '\n\n\n')

    if min_loss_for_saving > loss_valid:
        min_loss_for_saving = loss_valid
        print("Saved model with val_loss: ", loss_valid)
        model.save(model_path)

    del image_augmented
    del mask_augmented
    del bone_augmented

    print('---------------------------------------------------------------------------------------------')

    gc.collect()

In [ ]:
# Computing the metrics and saving the results

print("Loading the model")
model = tf.keras.models.load_model(model_path, custom_objects={'dice_metric_loss':dice_metric_loss})

prediction_train, bone_predict_train = model.predict(x_train, batch_size=4)
prediction_valid, bone_predict_valid = model.predict(x_valid, batch_size=4)
prediction_test, bone_predict_test = model.predict(x_test, batch_size=4)

print("Predictions done")

dice_train = f1_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                           np.ndarray.flatten(prediction_train > 0.5))
dice_test = f1_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                          np.ndarray.flatten(prediction_test > 0.5))
dice_valid = f1_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                           np.ndarray.flatten(prediction_valid > 0.5))

print("Dice finished")


miou_train = jaccard_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                           np.ndarray.flatten(prediction_train > 0.5))
miou_test = jaccard_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                          np.ndarray.flatten(prediction_test > 0.5))
miou_valid = jaccard_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                           np.ndarray.flatten(prediction_valid > 0.5))

print("Miou finished")


precision_train = precision_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                                  np.ndarray.flatten(prediction_train > 0.5))
precision_test = precision_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                                 np.ndarray.flatten(prediction_test > 0.5))
precision_valid = precision_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                                  np.ndarray.flatten(prediction_valid > 0.5))

print("Precision finished")


recall_train = recall_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                            np.ndarray.flatten(prediction_train > 0.5))
recall_test = recall_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                           np.ndarray.flatten(prediction_test > 0.5))
recall_valid = recall_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                            np.ndarray.flatten(prediction_valid > 0.5))

print("Recall finished")


accuracy_train = accuracy_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                                np.ndarray.flatten(prediction_train > 0.5))
accuracy_test = accuracy_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                               np.ndarray.flatten(prediction_test > 0.5))
accuracy_valid = accuracy_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                                np.ndarray.flatten(prediction_valid > 0.5))


print("Accuracy finished")


final_file = 'results_' + model_type + '_' + str(filters) + '_' + dataset_type + '_19_1.txt'
print(final_file)

with open(final_file, 'a') as f:
    f.write(dataset_type + '\n\n')
    f.write('dice_train: ' + str(dice_train) + ' dice_valid: ' + str(dice_valid) + ' dice_test: ' + str(dice_test) + '\n\n')
    f.write('miou_train: ' + str(miou_train) + ' miou_valid: ' + str(miou_valid) + ' miou_test: ' + str(miou_test) + '\n\n')
    f.write('precision_train: ' + str(precision_train) + ' precision_valid: ' + str(precision_valid) + ' precision_test: ' + str(precision_test) + '\n\n')
    f.write('recall_train: ' + str(recall_train) + ' recall_valid: ' + str(recall_valid) + ' recall_test: ' + str(recall_test) + '\n\n')
    f.write('accuracy_train: ' + str(accuracy_train) + ' accuracy_valid: ' + str(accuracy_valid) + ' accuracy_test: ' + str(accuracy_test) + '\n\n\n\n')

print('File done')

In [ ]:
print(dice_test)
print(miou_test)
print(precision_test)
print(recall_test)
print(accuracy_test)

In [ ]:
prediction_test, bone_predict_test = model.predict(x_test, batch_size=1)

ここで一つ一つの精度出せない？

In [ ]:
step=0

for i in prediction_test:
    print(step)
    step+=1
    img = np.array(i)
    plt.imshow(img)
    plt.show()